# Visualize and analyze the geospatial features of protests and armed conflicts using the georapid API hosted on Rapid API

* You need a valid Rapid API account
* The system environment must contain your Rapid API key ('x_rapidapi_key') and is send as a http header parameter.

In [ ]:
# author: Jan Tschada
# SPDX-License-Identifer: Apache-2.0
from arcgis import GIS
from arcgis.features import FeatureSet
from datetime import datetime, timedelta
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory
from georapid.formats import OutFormat
from georapid.protests import aggregate, articles, hotspots
from georapid.conflicts import aggregate as aggregate_conflicts, cluster as cluster_conflicts, query as query_conflicts

In [ ]:
help(EnvironmentClientFactory.create_client_with_host)

In [ ]:
host = "geoprotests.p.rapidapi.com"
client: GeoRapidClient = EnvironmentClientFactory.create_client_with_host(host)

In [ ]:
help(aggregate)

## Connect to ArcGIS Online anonymously

In [ ]:
gis = GIS()

## Use case: Visualize the news related to protests of 24th February 2022

In [ ]:
date_of_interest = datetime(2022, 2, 24)

world_map = gis.map()
world_map.basemap = 'dark-gray-vector'

protests_featureset = FeatureSet.from_dict(aggregate(client, date_of_interest, OutFormat.ESRI))
if protests_featureset.sdf.empty:
    print("The feature set is empty!")
else:
    protests_featureset.sdf.spatial.plot(world_map,
                                     renderer_type='c', # for class breaks renderer
                                     method='esriClassifyNaturalBreaks', # classification algorithm
                                     class_count=5, # choose the number of classes
                                     col='count', # numeric column to classify
                                     cmap='YlOrRd', # color map to pick colors from for each class
                                     alpha=0.35 # specify opacity
    )
world_map

## Use Case: Visualize the hotspots related to protests of 24th February 2022

In [ ]:
world_map = gis.map()
world_map.basemap = 'dark-gray-vector'

protests_featureset = FeatureSet.from_dict(hotspots(client, date_of_interest, OutFormat.ESRI))
if protests_featureset.sdf.empty:
    print("The feature set is empty!")
else:
    protests_featureset.sdf.spatial.plot(world_map,
                                     renderer_type='c', # for class breaks renderer
                                     method='esriClassifyNaturalBreaks', # classification algorithm
                                     class_count=5, # choose the number of classes
                                     col='count', # numeric column to classify
                                     cmap='YlOrRd', # color map to pick colors from for each class
                                     alpha=0.35 # specify opacity
    )
world_map

In [ ]:
protests_featureset.sdf

## Use Case: Visualize the armed conflicts of 24th February 2022

In [ ]:
host = "geoconflicts.p.rapidapi.com"
conflicts_client: GeoRapidClient = EnvironmentClientFactory.create_client_with_host(host)

In [ ]:
date_of_interest = datetime(2022, 2, 24)

world_map = gis.map()
world_map.basemap = 'dark-gray-vector'

conflicts_featureset = FeatureSet.from_dict(aggregate_conflicts(conflicts_client, date_of_interest, OutFormat.ESRI))
if conflicts_featureset.sdf.empty:
    print("The feature set is empty!")
else:
    conflicts_featureset.sdf.spatial.plot(world_map,
                                     renderer_type='c', # for class breaks renderer
                                     method='esriClassifyNaturalBreaks', # classification algorithm
                                     class_count=5, # choose the number of classes
                                     col='count', # numeric column to classify
                                     cmap='YlOrRd', # color map to pick colors from for each class
                                     alpha=0.35 # specify opacity
    )
world_map

In [ ]:
date_of_interest = datetime(2022, 2, 24)

world_map = gis.map()
world_map.basemap = 'dark-gray-vector'

cluster_conflicts_featureset = FeatureSet.from_dict(cluster_conflicts(conflicts_client, date_of_interest, OutFormat.ESRI))
if cluster_conflicts_featureset.sdf.empty:
    print("The feature set is empty!")
else:
    cluster_conflicts_featureset.sdf.spatial.plot(world_map,
                                     renderer_type='c', # for class breaks renderer
                                     method='esriClassifyNaturalBreaks', # classification algorithm
                                     class_count=5, # choose the number of classes
                                     col='events', # numeric column to classify
                                     cmap='YlOrRd', # color map to pick colors from for each class
                                     alpha=0.35 # specify opacity
    )
world_map

In [ ]:
conflict_events_featureset = FeatureSet.from_dict(query_conflicts(conflicts_client, date_of_interest, OutFormat.ESRI))
conflict_events_featureset.sdf.groupby('admin1')['location'].count().sort_values(ascending=False)

### Import the feature sets into ArcGIS Online

In [ ]:
from getpass import getpass
username = getpass('Username')
gis_admin = GIS(username=username)

In [ ]:
out_folder = ''
gis_admin.content.import_data(conflicts_featureset, title='Conflict Events Aggregated', folder=out_folder)

In [ ]:
gis_admin.content.import_data(cluster_conflicts_featureset, title='Conflict Events Clustered', folder=out_folder)

In [ ]:
gis_admin.content.import_data(conflict_events_featureset, title='Conflict Events', folder=out_folder)